# Image Detection with Tensorflow 1.x (GPU Enabled)


In [0]:
%tensorflow_version 1.x

# Enable GPU acceleration by navigating to 

> Edit > Notebook Settings > Choose "GPU" for hardware acceleration.

Then run the cell below to make sure the GPU is setup. You should see 'Found GPU'

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# **Mount** your google drive my using **'Mount drive'** from the files section.

Remember to change to the name of your folder. 

Setting the environment

In [13]:
%cd /content/drive/My Drive/RoadSignDetection/models/research/
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/RoadSignDetection/models/research/:/content/drive/My Drive/RoadSignDetection/models/research/slim'

/content/drive/My Drive/RoadSignDetection/models/research


# Imports

In [0]:
import numpy as np
import os
import tensorflow as tf
import cv2
import time
from google.colab.patches import cv2_imshow
import glob
import csv

In [0]:
# Import the object detection module.
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [16]:
%cd '/content/drive/My Drive/RoadSignDetection/models/research/object_detection'

/content/drive/My Drive/RoadSignDetection/models/research/object_detection


# Model Preparation

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

In [0]:
MODEL_NAME = 'inference_Final_RCNN_13'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# Load the Tensorflow model into memory.

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Loading label map
Label maps map indices to category names, so that when our convolution network predicts `2`, we know that this corresponds to `Speed limit (50km/h)`. Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [0]:
PATH_TO_LABELS = 'training/label_map.pbtxt'
NUM_CLASSES = 13

# Adding Labels and categories
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Main Object Detection Function

In [0]:
def ObjectDetection(imagePath, actualCategory):
  
  image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

  # Each box represents a part of the image where a particular object was detected.
  detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

  # Each score represent how level of confidence for each of the objects.
  # Score is shown on the result image, together with the class label.
  detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
  detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
  num_detections = detection_graph.get_tensor_by_name('num_detections:0')

  image = cv2.imread(imagePath)

  # Resize image with aspect ratio
  # With Min Dimensions:150 and Max Dimensions: 300
  resolutionDict = {}
  width = int(image.shape[1])
  height = int(image.shape[0])
  resolutionDict['width'] = width
  resolutionDict['height'] = height
  aspectRatio = height / width
  minResol = min(resolutionDict.items(), key=lambda x: x[1])
  maxResol = max(resolutionDict.items(), key=lambda x: x[1])

  # Resizing to 150 minimum dimenion
  # Whilst having a cap for the other dimension for 300
  if minResol[1] < 150:
    if minResol[0] == 'width':
      width = 150
      height = width * aspectRatio
      if height > 300:
        height = 300
    elif minResol[0] == 'height':
      height = 150
      width = height / aspectRatio
      if width > 300:
        width = 300
  
  # Resizing to 300 maximum dimenion
  # Whilst having a cap for the other dimension for 150
  elif maxResol[1] > 300:
    if maxResol[0] == 'width':
      width = 300
      height = width * aspectRatio
      if height < 150:
        height = 150
    if maxResol[0] == 'height':
      height = 300
      width = height / aspectRatio
      if width < 150:
        width = 150
  
  image = cv2.resize(image, (int(width), int(height)))

  # Resize image without aspect ratio
  # width = 150
  # height = 150
  # image = cv2.resize(image, (int(width), int(height)))

  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image, axis=0)

  # Actual detection.
  (boxes, scores, classes, num) = sess.run(
      [detection_boxes, detection_scores, detection_classes, num_detections],
      feed_dict={image_tensor: image_np_expanded})
  
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image,
      np.squeeze(boxes),
      np.squeeze(classes).astype(np.int32),
      np.squeeze(scores),
      category_index,
      use_normalized_coordinates=True, 
      line_thickness=3,
      min_score_thresh=.5)

  # Provides output of the category with higest score
  predictedClass = [category_index.get(i).get('name') for i in classes[0]][0]
  if predictedClass == actualCategory:
    cv2_imshow(image)
    return 1
  else:
    cv2_imshow(image)
    return 0

# Testing Images in the 'test_images' folder and getting model the performance metrics

In [0]:
%cd '/content/drive/My Drive/RoadSignDetection/models/research/object_detection'

/content/drive/My Drive/RoadSignDetection/models/research/object_detection


In [26]:
testImagesInfo = {}
with open('test_labels.csv', 'r') as currentfile:
  reader = csv.reader(currentfile)
  next(reader)
  for row in reader:
    testImagesInfo[row[0]] = row[3]

os.chdir('test_images')
TEST_IMAGE_PATHS = glob.glob('*.jpg')
TEST_IMAGE_PATHS.extend(glob.glob('*.png'))
correctDetections = 0
numImages = 0

startTime = time.time()

for imagePath in TEST_IMAGE_PATHS:
  numImages += 1
  correctDetections += ObjectDetection(imagePath, testImagesInfo.get(imagePath))

endTime = time.time()

print('FPS: ', round(numImages/(endTime - startTime),2))
print('Speed(ms): ',  round(((endTime - startTime)/numImages)*1000,2))
print('Accuracy: ', round(correctDetections/numImages * 100,2),'%')
os.chdir("..")

Output hidden; open in https://colab.research.google.com to view.